In [163]:
import numpy as np
import pandas as pd
import os
from Levenshtein import ratio, distance
from datetime import datetime
from dateutil.relativedelta import relativedelta

Import the the deal data and URL data.

In [1]:
def find(name, path):
    for root, dirs, files in os.walk(path):
        if name in files:
            return os.path.join(root, name)

In [274]:
path = find('hmspall.xlsx', 'C://')
transdata = pd.read_excel(path)

In [317]:
path  = find('AllUrls.csv', 'C://')
allurls = pd.read_csv(path)
allurls['Date Effective'] = pd.to_datetime(allurls['Date Effective'], format='%d/%m/%Y %H:%M')
allurls = allurls[allurls['Date Effective']<'2018-01-01']
allurls = allurls.reset_index(drop=True)

In [318]:
allurls = allurls.rename(columns={'Unnamed: 4':'Target URL','Unnamed: 8':'Acquiror URL'})

Get the company names and find the effective date that belongs to the companies.

In [304]:
targets =list(transdata['Target Name'])
acquirors = list(transdata['Acquiror Name'])

In [305]:
double = list()
for index, row in allurls.iterrows():
    target = allurls.iloc[index]['Target Name']
    acquiror = allurls.iloc[index]['Acquiror Name']
    date = allurls.iloc[index]['Date Effective']
    transdatasub=transdata[(transdata['Target Name']==target)&(transdata['Acquiror Name']==acquiror)&
            (transdata['Date Effective']==date)]
    if transdatasub.shape[0]>1:
        double.append(index)

    if distance(target, acquiror)==1:
        double.append(index)

In [306]:
allurls = allurls.drop(index=[308, 564])
allurls = allurls.reset_index(drop=True)

In [307]:
allurls['Target URL'] = allurls['Target URL'].fillna('None')
allurls['Acquiror URL'] = allurls['Acquiror URL'].fillna('None')

Check if a URL is available and make them all in the correct format.

In [308]:
for index, row in allurls.iterrows():
    if allurls.iloc[index]['Target URL'] != 'None':
        if (allurls.iloc[index]['Target URL'].startswith('http')==False):
            new_url = 'https://' + allurls.iloc[index]['Target URL']
            allurls.at[index, 'Target URL'] = new_url
        if ('www' not in '/'.join(allurls.iloc[index]['Target URL'].split('/', 2)[-1:])) and (allurls.iloc[index]['Target URL'] != 'None'):
            new_url = 'https://www.'+'/'.join(allurls.iloc[index]['Target URL'].split('/',2)[2:])
            allurls.at[index, 'Target URL'] = new_url
    if allurls.iloc[index]['Acquiror URL'] != 'None':
        if (allurls.iloc[index]['Acquiror URL'].startswith('http')==False):
            new_url = 'https://' + allurls.iloc[index]['Acquiror URL']
            allurls.at[index, 'Acquiror URL'] = new_url
        if ('www' not in '/'.join(allurls.iloc[index]['Acquiror URL'].split('/', 2)[-1:])):
            new_url = 'https://www.'+'/'.join(allurls.iloc[index]['Acquiror URL'].split('/',2)[2:])
            allurls.at[index, 'Acquiror URL'] = new_url

In [319]:
allurls.columns

Index(['Date Effective', 'Target Name', 'Unnamed: 2', 'Unnamed: 3',
       'Target URL', 'Acquiror Name', 'Unnamed: 6', 'Unnamed: 7',
       'Acquiror URL'],
      dtype='object')

Create a dataframe that summarizes the name and date information.

In [313]:
companydates = pd.DataFrame(columns=['Date Effective', 'Target Name', 'Target URL',
                               'Target Companydate Before', 'Target Companydate After',
                               'Acquiror Name', 'Acquiror URL',
                              'Acquiror Companydate Before', 'Acquiror Companydate After'])

In [314]:
for index, row in allurls.iterrows():
    targeturl = row['Target URL']
    acquirorurl = row['Acquiror URL']
    target = row['Target Name']
    acquiror = row['Acquiror Name']
   
    date_effective = row['Date Effective']
    date_one_week_prior = date_effective + relativedelta(weeks=-1)
    date_one_week_prior = str(date_one_week_prior).replace('-',"")[:-9]
    date_two_years_after = date_effective + relativedelta(years=+2)
    date_two_years_after = str(date_two_years_after).replace('-',"")[:-9]
    targetfromurl = '/'.join(targeturl.split('.',2)[1:2])
    acquirorfromurl = '/'.join(acquirorurl.split('.',2)[1:2])
    if (targeturl != 'None') and (acquirorurl !='None'):
        targetbefore = targetfromurl + date_one_week_prior
        targetafter = targetfromurl +date_two_years_after
        acquirorbefore = acquirorfromurl +date_one_week_prior
        acquirorafter = acquirorfromurl +date_two_years_after
        companydates = companydates.append({'Date Effective':date_effective, 'Target Name':targetfromurl, 
                            'Target URL':targeturl,'Target Companydate Before':targetbefore, 
                            'Target Companydate After':targetafter,'Acquiror Name':acquirorfromurl, 
                            'Acquiror URL':acquirorurl,'Acquiror Companydate Before':acquirorbefore, 
                            'Acquiror Companydate After':acquirorafter}, ignore_index=True)
    elif (targeturl != 'None') and (acquirorurl == 'None'):
        targetbefore = targetfromurl + date_one_week_prior
        targetafter = targetfromurl +date_two_years_after
        companydates = companydates.append({'Date Effective':date_effective, 'Target Name':targetfromurl, 
                            'Target URL':targeturl,'Target Companydate Before':targetbefore, 
                            'Target Companydate After':targetafter,'Acquiror Name':acquiror, 
                            'Acquiror URL':'None','Acquiror Companydate Before':'None', 
                            'Acquiror Companydate After':'None'}, ignore_index=True)
    elif (targeturl == 'None') and (acquirorurl !='None'):
        acquirorbefore = acquirorfromurl +date_one_week_prior
        acquirorafter = acquirorfromurl +date_two_years_after
        companydates = companydates.append({'Date Effective':date_effective, 'Target Name':target, 
                            'Target URL':'None','Target Companydate Before':'None', 
                            'Target Companydate After':'None','Acquiror Name':acquirorfromurl, 
                            'Acquiror URL':acquirorurl,'Acquiror Companydate Before':acquirorbefore, 
                            'Acquiror Companydate After':acquirorafter}, ignore_index=True)
    elif (targeturl == 'None') and (acquirorurl =='None'):
        companydates = companydates.append({'Date Effective':date_effective, 'Target Name':target, 
                            'Target URL':'None','Target Companydate Before':'None', 
                            'Target Companydate After':'None','Acquiror Name':acquiror, 
                            'Acquiror URL':'None','Acquiror Companydate Before':'None', 
                            'Acquiror Companydate After':'None'}, ignore_index=True)

In [315]:
companydates.to_excel('C://AllCompanyDates.xlsx')